In [1]:
#importamos las librerias necesarias
import pickle
import numpy as np
import pandas as pd
import reverse_geocode
from joblib import Parallel, delayed
import time


In [2]:
# Cargar el archivo pickle (.pkl)
file_path = '/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/bussines_pkl/business.pkl'
df_bussines_pkl = pd.read_pickle(file_path)

In [3]:
#Eliminamos las columnas duplicadas del DataFrame copia
df_bussines_pkl = df_bussines_pkl.iloc[:, :14]

In [5]:
#vuelvo a analizar los valores nulos
print(df_bussines_pkl.isnull().sum())

business_id         0
name                0
address             0
city                0
state               3
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64


### Estudio de las columnas con nulos: 

<u>attributes:</u> Es un conjunto de atributos adicionales del negocio. Puede ser una cadena de texto, un diccionario u otro tipo de dato que contenga información adicional como si el negocio tiene Wi-Fi, accesibilidad para discapacitados, etc. <span style='color:lightpink'>consideramos que no es necesaria a nuestros intereses</span><span style='color:red'> SE PROCEDE A BORRAR LA COLUMNA</span>

<u>categories:</u> Son las categorías a las que pertenece el negocio. Es una cadena de texto que puede contener múltiples categorías separadas por comas, como "restaurante, italiano, pizza".<span style='color:lightblue'>consideramos que puede ser necesaria a nuestros intereses</span><span style='color:Gold'> SE ANALIZARA COMO RELLENAR LOS NULOS(103)</span>

<u>hours:</u> Es el horario de apertura del negocio. Puede ser una cadena de texto, un diccionario u otro tipo de dato que especifique los horarios de apertura y cierre para cada día de la semana. <span style='color:lightpink'>consideramos que no es necesaria a nuestros intereses</span><span style='color:red'> SE PROCEDE A BORRAR LA COLUMNA</span>

In [6]:
#Dropeamos las columnas 'attributes' y 'hours' de acuerdo al analisis previo:

df_bussines_pkl = df_bussines_pkl.drop(['attributes', 'hours'], axis=1)

In [7]:
#filtrar todo el dataframe por 'restaurants' en la columna 'categories'
mask = df_bussines_pkl['categories'].str.contains('Restaurants', case=False, na=False)
df_bussines_pkl = df_bussines_pkl[mask]

In [8]:
#funcion para obtener la cantidad de nucleos de la maquina, la cual se utilizara para la paralelizacion, dejando un nucleo libre para otras tareas(n-1)

import multiprocessing

# Obtener el número total de núcleos de procesamiento lógicos en la máquina
total_cores = multiprocessing.cpu_count()

# Decidir cuántos núcleos utilizar. Por ejemplo, usar todos menos uno para dejar recursos libres para otras tareas.
num_cores_to_use = total_cores - 1

# Luego, pasas este número al parámetro num_cores en la llamada a parallel_apply
#df_bussines_pkl[['reverse_city', 'reverse_state']] = parallel_apply(df_bussines_pkl, get_city_state, num_cores=num_cores_to_use)

In [10]:
#funcion para obtener la ciudad y el estado usando reverse_geocode, creando dos columnas nuevas en el dataframe: 'city_reverse', 'state_reverse'

# Cache de resultados
cache = {}

# Función para obtener la ciudad y el estado usando reverse_geocode
def get_city_state(lat, lon):
    coordinates = (lat, lon)
    if coordinates not in cache:
        location = reverse_geocode.search([coordinates])[0]
        cache[coordinates] = (location['city'], location['state'])
    return cache[coordinates]

# Paralelizar las solicitudes de geocodificación
def parallel_apply(df, func, num_cores=4):
    results = Parallel(n_jobs=num_cores)(delayed(func)(row['latitude'], row['longitude']) for idx, row in df.iterrows())
    return pd.DataFrame(results, columns=['city_reverse', 'state_reverse'])

# Medir el tiempo de ejecución
start_time = time.time()

# Aplicar la función en paralelo
df_bussines_pkl[['reverse_city', 'reverse_state']] = parallel_apply(df_bussines_pkl, get_city_state)

# # Mostrar el tiempo de ejecución si se desea
# print(f"--- {time.time() - start_time} seconds ---")


In [11]:
#borro las columnas de 'City', 'State', pues no se corresponden entre si, generando muchos conflictos
df_bussines_pkl = df_bussines_pkl.drop(['city', 'state'], axis=1)

In [12]:
# filtrar todo el dataframe por las ciudades de 'Pennsylvania', 'Texas', New York', 'California', 'Florida' de la columna 'reverse_state'
mask = df_bussines_pkl['reverse_state'].isin(['Pennsylvania', 'Texas', 'New York', 'California', 'Florida'])
df_bussines_pkl = df_bussines_pkl[mask]

In [13]:
#hago un csv para ver la estructura del archivo
df_bussines_pkl.to_csv('Yelp/notebooks_yelp/bussines_pkl/df_bussines_pkl_con_ciudad_estado.csv/df_bussines_pkl_con_ciudad_estado.csv', index=False)